# Twitter API v1.1 Client

In [2]:
import tweepy
import pandas as pd 
import numpy as np
import datetime 

## Autentication

In [3]:
def get_auth_twitter():

        consumer_key= 'drhztkGNhICDt3cwBGXi7zzQk'
        consumer_secret= 'FDeCBlSi4qIbR9oPsf3tphQjCO6HqZfYANWceYIdJDZHsOaabi'
        access_token= '207778119-E9W1kqRXBbNeU2n8vcA0OoMn9XZpE9yvmZrD8gUx'
        access_token_secret= 'dncbYQQrh5jlJ97OmgxeaGgqOFiC7jnNpXdPngWMCRtDE'
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

        return api


## Query API V1

In [139]:
def search_for_tweets(search_words, until, lang, items):

        ##search_words = "DisneyPlus"
        ##until= "2020-11-03"
        ##lang = 'pt'

        cursor = tweepy.Cursor(api.search,
                    q=search_words,
                    lang=lang,
                    until= until).items(items)

        return cursor

## Cursor to List

In [140]:
def iterator_to_list(cursor):

    return list(cursor)      

In [254]:
all_tweets_list[0]._json['in_reply_to_status_id'] = 0

In [266]:
all_tweets_list[4]._json['quoted_status_id']

1326570384032886784

## Creating a DF

In [256]:

def tweets_to_dataframe(tweets_list):

    columns = set()
    
    tweets_data = []

    
    for tweet in tweets_list:
        keys = tweet._json.keys()
        single_data = {}
        
        
        for k in keys:

            valor = tweet._json.get(k)

            if tweet._json.get(k) == None:

                try:

                    single_data[k] = 0

                except Exception as e:
                    print(e)
                
            
            else:
              
               try:
                    single_data[k] = tweet._json.get(k)
               
               except Exception as e:
                    print(e)
                    single_data[k] = 0

                
            
            columns.add(k)

        tweets_data.append(single_data)    
    

    header_cols = list(columns)
    df = pd.DataFrame(tweets_data, columns=header_cols)
    df.head()
    
    return df

## Extracting data from objects

Entities column: finding mentions

In [327]:
def mentions(df):

        df['screen_name_mention_1'] = df['entities'].apply(lambda x: x['user_mentions'][0]['screen_name'] if x['user_mentions'] else None)

        df['id_mention_1'] = df['entities'].apply(lambda x: x['user_mentions'][0]['id_str'] if x['user_mentions'] else 0)

        return df

Retweeted_Status - Finding the author of the original tweet

In [273]:
def retweets(df):

    df['retweeted_from_screen_name'] = df['retweeted_status'].apply(lambda x: x['user']['screen_name'] if x is not np.nan else None)

    df['retweeted_from_id'] = df['retweeted_status'].apply(lambda x: x['user']['id_str'] if x is not np.nan else 0)

    df['retweeted_from_is_verified'] = df['retweeted_status'].apply(lambda x: x['user']['verified'] if x is not np.nan else None)

    return df 

User - Information about the user

In [274]:
def user(df):

    df['user_screen_name'] = df['user'].apply(lambda x: x['screen_name'] if x is not np.nan else None)

    df['user_id'] = df['user'].apply(lambda x: x['id_str'] if x is not np.nan else 0)

    df['user_followers'] = df['user'].apply(lambda x: x['followers_count'] if x is not np.nan else None)

    df['user_following'] = df['user'].apply(lambda x: x['friends_count'] if x is not np.nan else None)

    df['user_is_verified'] = df['user'].apply(lambda x: x['verified'] if x is not np.nan else None)

    return df

In [328]:
## New Columns

def destrinchando_colunas(df):
    
    mentions(df)
    retweets(df)
    user(df)

    return df


## Formating Datetime

In [314]:
def format_created_at(df):

    df['created_at'] = df['created_at'].apply(lambda x: datetime.strftime(datetime.strptime(x,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S'))

    return df

## Creating files

In [64]:
def create_csv(df, file_name):
    
    df.to_csv(f'C:/Users/amand/Documents/Jupyter Notebooks/DisneyPlus/Data/{file_name}.csv')

In [117]:
def create_pickle(df, file_name):

    df.to_pickle(f'C:/Users/amand/Documents/Jupyter Notebooks/DisneyPlus/Data/{file_name}.pkl')

# Testing Functions

Raw Data

In [286]:
api = get_auth_twitter()

In [329]:
all_tweets = search_for_tweets("DisneyPlus", "2020-11-04", 'pt', 100)

In [330]:
all_tweets_list = iterator_to_list(all_tweets)

DataFrame

In [331]:
df= tweets_to_dataframe(all_tweets_list)

In [332]:
df2 = destrinchando_colunas(df)

## Selecting Columns

In [124]:
graph_columns_ids = ['user_id', 'retweeted_from_id' , 'id_mention_1', 'in_reply_to_user_id',  'id']

In [291]:
important_columns = ['user_screen_name', 'user_id', 'user_is_verified', 'retweeted_from_screen_name', 'retweeted_from_id', 'retweeted_from_is_verified', 'screen_name_mention_1' , 'id_mention_1', 'in_reply_to_screen_name', 'in_reply_to_user_id', 'created_at', 'id', 'text']

In [292]:
df3 = df2[graph_columns_ids]

In [333]:
df4 = df2[important_columns]

In [334]:
df4.created_at

0     Tue Nov 03 23:58:55 +0000 2020
1     Tue Nov 03 23:58:34 +0000 2020
2     Tue Nov 03 23:58:21 +0000 2020
3     Tue Nov 03 23:58:12 +0000 2020
4     Tue Nov 03 23:57:51 +0000 2020
                   ...              
95    Tue Nov 03 23:04:52 +0000 2020
96    Tue Nov 03 23:04:49 +0000 2020
97    Tue Nov 03 23:02:59 +0000 2020
98    Tue Nov 03 23:02:46 +0000 2020
99    Tue Nov 03 23:02:34 +0000 2020
Name: created_at, Length: 100, dtype: object

In [335]:
df4 = format_created_at(df4)

In [304]:
df4['created_at'] = df4['created_at'].apply(lambda x: datetime.strftime(datetime.strptime(x,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S'))

In [308]:
import matplotlib.pyplot as plt

In [336]:
df4

,user_screen_name,user_id,user_is_verified,retweeted_from_screen_name,retweeted_from_id,retweeted_from_is_verified,screen_name_mention_1,id_mention_1,in_reply_to_screen_name,in_reply_to_user_id,created_at,id,text
0,Jose_Navas_Jr,32901996,False,vcfaztv,16354038,False,vcfaztv,16354038,0,0,2020-11-03 23:58:55,1323776705912315911,RT @vcfaztv: Os serviços de streaming Globopla...
1,zucatellidanilo,2978733862,False,globoplay,55569724,True,globoplay,55569724,0,0,2020-11-03 23:58:34,1323776617806704643,RT @globoplay: ✨Os sonhos são reais✨\nQuer ser...
2,WilliamsHenriq,1462821571,False,globoplay,55569724,True,globoplay,55569724,0,0,2020-11-03 23:58:21,1323776560634175490,RT @globoplay: ✨Os sonhos são reais✨\nQuer ser...
3,Karolmebaby,1025340762303750144,False,ManulovesKs,1204569627201855489,False,ManulovesKs,1204569627201855489,0,0,2020-11-03 23:58:12,1323776525058084864,"RT @ManulovesKs: Só pago 27,90 no Disney Plus ..."
4,ally_blu,225906556,False,globoplay,55569724,True,globoplay,55569724,0,0,2020-11-03 23:57:51,1323776436445085698,"RT @globoplay: 💫""Qualquer coisa que seu coraçã..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,eudahan,4330786047,False,jurandirfilho,8703862,False,jurandirfilho,8703862,0,0,2020-11-03 23:04:52,1323763102727442433,RT @jurandirfilho: A @DisneyPlusBR vai chegar ...
96,LovztoRuins,2777259895,False,BwayMeme,3048169227,False,BwayMeme,3048169227,0,0,2020-11-03 23:04:49,1323763089171451905,RT @BwayMeme: 🚨O Disney+ (com seu vasto catalo...
97,YonahSantos1,1323508103732727809,False,DisneyPlusBR,1011372006753746944,True,DisneyPlusBR,1011372006753746944,0,0,2020-11-03 23:02:59,1323762629878435840,RT @DisneyPlusBR: A vida é cheia de possibilid...
98,whoiswenz,795802693952565248,False,None,0,None,None,0,0,0,2020-11-03 23:02:46,1323762573695746051,GLOBOPLAY COM DISNEYPLUS QUE HISTORIA É ESSA


In [337]:
create_pickle(df4,'until_04_11')

In [338]:
create_csv(df4, 'until_04_11')